In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
df_coor = pd.read_csv("C:/Users/hejia/Desktop/DS_projects/TorontaData.csv")

In [ ]:
df_coor["Borough"].value_counts().sum

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [38]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coor['Latitude'], df_coor['Longitude'], df_coor['Borough'], df_coor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

let us explore the neighborhoods in North York of Toronto

In [ ]:
n_york_data = df_coor[df_coor['Borough'] == 'North York'].reset_index(drop=True)
n_york_data.head()

In [ ]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of North York using latitude and longitude values
map_n_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(n_york_data['Latitude'], n_york_data['Longitude'], n_york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_n_york)  
    
map_n_york

In [ ]:
# define Foursquare Credentials and Version
CLIENT_ID = '3TJILUL33Y122CMLX3WN4H245RQZHXPF13MRJ5UWKTG4VS1Z' # your Foursquare ID
CLIENT_SECRET = 'RS2ZHMTYLMYOA40SHDRXXXXD521CGDDA1ZW1M3ZT3P15VLB5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [ ]:
# create a function to repert exploring all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

n_york_venues = getNearbyVenues(names=n_york_data['Neighborhood'],
                                   latitudes=n_york_data['Latitude'],
                                   longitudes=n_york_data['Longitude']
                                  )

In [ ]:
n_york_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(n_york_venues['Venue Category'].unique())))

In [ ]:
# Analyze each Neighborhood
# one hot encoding
n_york_onehot = pd.get_dummies(n_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
n_york_onehot['Neighborhood'] = n_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [n_york_onehot.columns[-1]] + list(n_york_onehot.columns[:-1])
n_york_onehot = n_york_onehot[fixed_columns]

n_york_onehot.head()

In [ ]:
n_york_onehot.shape

In [ ]:
n_york_grouped = n_york_onehot.groupby('Neighborhood').mean().reset_index()
n_york_grouped

In [ ]:
n_york_grouped.shape

In [ ]:
num_top_venues = 5

for hood in n_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = n_york_grouped[n_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = n_york_grouped['Neighborhood']

for ind in np.arange(n_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(n_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


In [ ]:
# set number of clusters
kclusters = 4

n_york_grouped_clustering = n_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(n_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

n_york_merged = n_york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
n_york_merged = n_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

n_york_merged # check the last columns!

In [ ]:
n_york_merged = n_york_merged.dropna() 
n_york_merged

In [ ]:
import numpy as np
import sys

np.set_printoptions(threshold=sys.maxsize)   
n_york_merged = n_york_merged.astype({'Cluster Labels': int})
n_york_merged

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [37]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(n_york_merged['Latitude'], n_york_merged['Longitude'], n_york_merged['Neighborhood'], n_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

 
map_clusters